In [ ]:
!poetry run jupyter-notebook

In [1]:
!pwd

/Users/jasperaugusttootsi/projects/nanomed/Enrichment/notebooks


In [2]:
# Step 1: Import Necessary Libraries
import os
import pandas as pd
from Bio.Seq import Seq
from Bio import SeqIO
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# Directories
UPLOAD_DIR = "../backend/uploaded_files/"
PARSED_DIR = "../backend/parsed_data/"
TRIMMED_DIR = "../backend/trimmed_data/"
EXTRACTED_ORFS_DIR = "../backend/extracted_orfs/"

os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(PARSED_DIR, exist_ok=True)
os.makedirs(TRIMMED_DIR, exist_ok=True)
os.makedirs(EXTRACTED_ORFS_DIR, exist_ok=True)


# Parameters
seed_peptide = "MLGDPNS"  # User-provided seed sequence (amino acids)
quality_cutoff = 20  # User-defined quality cutoff
# adapter_sequence = "AGATCGGAAGAGCACACGTCTGAACTCCAGTCA"  # Example adapter sequence
trim_length = 100  # Number of nucleotides to include downstream from the seed


In [3]:
def dna2aa(sequence):
    """

    translate dna sequence into aminoacid sequence

    assumes codons start at first character

    if last part is shorter than 3 nucleotides it is ignored

    if other letters than ATGC in codon, codon is skipped!

    """

    gencode = {
        "ATA": "I",
        "ATC": "I",
        "ATT": "I",
        "ATG": "M",
        "ACA": "T",
        "ACC": "T",
        "ACG": "T",
        "ACT": "T",
        "AAC": "N",
        "AAT": "N",
        "AAA": "K",
        "AAG": "K",
        "AGC": "S",
        "AGT": "S",
        "AGA": "R",
        "AGG": "R",
        "CTA": "L",
        "CTC": "L",
        "CTG": "L",
        "CTT": "L",
        "CCA": "P",
        "CCC": "P",
        "CCG": "P",
        "CCT": "P",
        "CAC": "H",
        "CAT": "H",
        "CAA": "Q",
        "CAG": "Q",
        "CGA": "R",
        "CGC": "R",
        "CGG": "R",
        "CGT": "R",
        "GTA": "V",
        "GTC": "V",
        "GTG": "V",
        "GTT": "V",
        "GCA": "A",
        "GCC": "A",
        "GCG": "A",
        "GCT": "A",
        "GAC": "D",
        "GAT": "D",
        "GAA": "E",
        "GAG": "E",
        "GGA": "G",
        "GGC": "G",
        "GGG": "G",
        "GGT": "G",
        "TCA": "S",
        "TCC": "S",
        "TCG": "S",
        "TCT": "S",
        "TTC": "F",
        "TTT": "F",
        "TTA": "L",
        "TTG": "L",
        "TAC": "Y",
        "TAT": "Y",
        "TAA": "_",
        "TAG": "_",
        "TGC": "C",
        "TGT": "C",
        "TGA": "_",
        "TGG": "W",
    }

    return "".join(
        [
            gencode[sequence[n : n + 3]]
            for n in range(0, len(sequence), 3)
            if sequence[n : n + 3] in gencode
        ]
    )

fs = "ACGTMRYKVHDBU"

ts = "TGCAKYRMBDHVT"

global ttable

ttable = str.maketrans(fs, ts)


def reverse_complement(sequence):
    """Reverse complement



    Returns the reverse complement sequence.

    Knows alternative IUPAC nucleotides. New string object."""

    sequence = sequence.upper()

    sequence = sequence[-1::-1].translate(ttable)  # why first '-1'? for magic reasons

    return sequence

In [4]:
import subprocess
from Bio import SeqIO


def run_fastp(input_fastq, output_fastq, quality_cutoff=20):
    cmd = [
        "fastp",
        "-i",
        input_fastq,
        "-o",
        output_fastq,
        "--detect_adapter_for_pe",
        "-q",
        str(quality_cutoff),
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"fastp completed. Trimmed file saved to {output_fastq}")
    except subprocess.CalledProcessError as e:
        print(f"fastp failed: {e}")


def import_fastq(file_path: str) -> pd.DataFrame:
    """
    Parses a FASTQ file into a pandas DataFrame.

    :param file_path: Full path to the FASTQ file.
    :return: DataFrame with 'Sequence_ID' and 'Sequence' columns.
    """
    records = []
    try:
        with open(file_path, "r") as handle:
            for record in SeqIO.parse(handle, "fastq"):
                records.append({"Sequence_ID": record.id, "Sequence": str(record.seq)})
    except Exception as e:
        logging.error(f"Error parsing {file_path}: {e}")
        return pd.DataFrame()

    df = pd.DataFrame(records)
    parsed_file = os.path.join(PARSED_DIR, f"{os.path.splitext(os.path.basename(file_path))[0]}.csv")
    df.to_csv(parsed_file, index=False)
    logging.info(f"Parsed {file_path} into {parsed_file}")
    return df


def find_seed_positions(sequence: str, seed_peptide: str) -> list:
    """Find positions where seed peptide occurs in any ORF."""
    positions = []
    sequence = sequence.upper()

    # Look in forward and reverse strands
    strands = [sequence, reverse_complement(sequence)]

    for strand_idx, strand in enumerate(strands):
        # Check each reading frame (0, 1, 2)
        for frame in range(3):
            # Get subsequence starting at this frame
            subseq = strand[frame:]
            # Ensure length is divisible by 3
            subseq = subseq[: len(subseq) - (len(subseq) % 3)]

            # Translate to protein using our dna2aa function
            protein = dna2aa(subseq)

            # Find seed peptide occurrences
            start = 0
            while True:
                pos = protein.find(seed_peptide, start)
                if pos == -1:
                    break

                # Convert protein position to DNA position
                dna_pos = pos * 3 + frame

                # Adjust position for reverse complement
                if strand_idx == 1:
                    dna_pos = len(sequence) - dna_pos - len(seed_peptide) * 3

                positions.append(
                    {
                        "position": dna_pos,
                        "frame": frame,
                        "strand": "-" if strand_idx == 1 else "+",
                    }
                )
                start = pos + 1

    return positions


def extract_orf(sequence: str, start_pos: int, trim_length: int = 100) -> str:
    """
    Extracts the ORF starting from the seed position, trimmed to the specified length.

    :param sequence: Full DNA sequence.
    :param start_pos: Start position of the seed within the sequence.
    :param trim_length: Number of nucleotides to include downstream from the seed.
    :return: Extracted ORF sequence.
    """
    end_pos = start_pos + trim_length
    return (
        sequence[start_pos:end_pos]
        if end_pos <= len(sequence)
        else sequence[start_pos:]
    )


def search_and_extract_seeds(
    df: pd.DataFrame, seed_peptide: str, trim_length: int = 100
) -> pd.DataFrame:
    """Searches all reading frames and orientations for the seed peptide sequence."""
    extracted_data = {
        "Sequence_ID": [],
        "Original_Sequence": [],
        "DNA_Sequence": [],
        "Peptide": [],
        "Frame": [],
        "Orientation": [],
    }

    for _, row in df.iterrows():
        sequence = row["Sequence"]
        seed_positions = find_seed_positions(sequence, seed_peptide)

        for pos_data in seed_positions:
            dna_pos = pos_data["position"]
            frame = pos_data["frame"]
            strand = pos_data["strand"]

            # Extract DNA sequence
            if strand == "+":
                dna_seq = sequence[dna_pos : dna_pos + trim_length]
            else:
                dna_seq = reverse_complement(
                    sequence[max(0, dna_pos - trim_length) : dna_pos]
                )

            # Ensure sequence length is divisible by 3
            dna_seq = dna_seq[: len(dna_seq) - (len(dna_seq) % 3)]

            # Translate to peptide
            peptide = dna2aa(dna_seq)

            if peptide and peptide.startswith(seed_peptide):
                # Remove seed sequence
                peptide_without_seed = peptide[len(seed_peptide) :]

                # Stop at first stop codon
                stop_pos = peptide_without_seed.find("_")
                if stop_pos != -1:
                    peptide_without_seed = peptide_without_seed[:stop_pos]

                # Only save if we have something after removing seed and stopping at stop codon
                if peptide_without_seed:
                    extracted_data["Sequence_ID"].append(row["Sequence_ID"])
                    extracted_data["Original_Sequence"].append(sequence)
                    extracted_data["DNA_Sequence"].append(dna_seq)
                    extracted_data["Peptide"].append(peptide_without_seed)
                    extracted_data["Frame"].append(frame)
                    extracted_data["Orientation"].append(strand)

    return pd.DataFrame(extracted_data)


def process_fastq_file(file_name: str):
    """
    Processes a single FASTQ file: imports, trims adapters, filters quality, and extracts ORFs.

    :param file_name: Name of the FASTQ file.
    :return: DataFrame with extracted peptides.
    """
    logging.info(f"Processing file: {file_name}")

    # Extract tissue/organ name from filename (assuming format: tissue_other_info.fastq)
    tissue = file_name.split("_")[0]

    # Full paths
    input_file = os.path.join(UPLOAD_DIR, file_name)
    trimmed_file = os.path.join(
        TRIMMED_DIR, f"{os.path.splitext(file_name)[0]}_trimmed.fastq"
    )

    # Run fastp for quality trimming
    run_fastp(input_file, trimmed_file, quality_cutoff=quality_cutoff)

    # Import and parse the trimmed FASTQ file
    df_parsed = import_fastq(trimmed_file)

    if df_parsed is None or df_parsed.empty:
        logging.warning(f"No sequences parsed from {file_name}")
        return None

    # Extract peptides containing seed sequence
    extracted_df = search_and_extract_seeds(
        df_parsed, seed_peptide=seed_peptide, trim_length=trim_length
    )

    if extracted_df is not None and not extracted_df.empty:
        # Add tissue/organ source column
        extracted_df["Tissue"] = tissue

    return extracted_df


def plot_results(combined_df):
    plt.style.use("ggplot")
    fig = plt.figure(figsize=(15, 12))

    ax1 = plt.subplot(321)
    frame_orient_counts = combined_df.groupby(["Frame", "Orientation"]).size().unstack()
    frame_orient_counts.plot(kind="bar", ax=ax1)
    ax1.set_title("Distribution of Reading Frames by Orientation")
    ax1.set_xlabel("Frame")
    ax1.set_ylabel("Count")

    ax2 = plt.subplot(322)
    combined_df["Peptide_Length"] = combined_df["Peptide"].str.len()
    sns.histplot(data=combined_df, x="Peptide_Length", bins=30, ax=ax2)
    ax2.set_title("Distribution of Peptide Lengths")
    ax2.set_xlabel("Length (aa)")

    ax3 = plt.subplot(323)
    seq_counts = combined_df["Peptide"].value_counts().head(10)
    seq_counts.plot(kind="bar", ax=ax3)
    ax3.set_title("Top 10 Most Common Peptides")
    ax3.set_xlabel("Peptide")
    plt.xticks(rotation=45, ha="right")

    ax4 = plt.subplot(324)
    tissue_counts = combined_df["Tissue"].value_counts()
    tissue_counts.plot(kind="bar", ax=ax4)
    ax4.set_title("Peptide Distribution by Tissue")
    ax4.set_xlabel("Tissue")
    ax4.set_ylabel("Count")
    plt.xticks(rotation=45, ha="right")

    ax5 = plt.subplot(325)
    tissue_frame_counts = combined_df.groupby(["Tissue", "Frame"]).size().unstack()
    tissue_frame_counts.plot(kind="bar", stacked=True, ax=ax5)
    ax5.set_title("Reading Frame Distribution by Tissue")
    ax5.set_xlabel("Tissue")
    ax5.set_ylabel("Count")
    plt.xticks(rotation=45, ha="right")

    ax6 = plt.subplot(326)
    tissue_orient_counts = (
        combined_df.groupby(["Tissue", "Orientation"]).size().unstack()
    )
    tissue_orient_counts.plot(kind="bar", stacked=True, ax=ax6)
    ax6.set_title("Orientation Distribution by Tissue")
    ax6.set_xlabel("Tissue")
    ax6.set_ylabel("Count")
    plt.xticks(rotation=45, ha="right")

    plt.tight_layout()
    plt.show()

    print("\nSummary Statistics:")
    print(f"Total peptides found: {len(combined_df)}")
    print(f"Unique peptides: {combined_df['Peptide'].nunique()}")
    print("\nTissue distribution:")
    print(combined_df["Tissue"].value_counts())
    print("\nOrientation distribution by tissue:")
    print(pd.crosstab(combined_df["Tissue"], combined_df["Orientation"]))
    print("\nFrame distribution by tissue:")
    print(pd.crosstab(combined_df["Tissue"], combined_df["Frame"]))

In [9]:
from joblib import Parallel, delayed

# Process All FASTQ Files in Parallel
fastq_files = [
    os.path.join(UPLOAD_DIR, f)
    for f in os.listdir(UPLOAD_DIR)
    if f.endswith((".fastq", ".fq"))
]


# Process files in parallel
results = Parallel(n_jobs=-1, verbose=1)(
    delayed(process_fastq_file)(os.path.basename(file_path))
    for file_path in fastq_files
)

# Save each result separately
for df in results:
    if df is not None and not df.empty:
        tissue = df["Tissue"].iloc[0]  # Get tissue name from first row
        output_file = os.path.join(EXTRACTED_ORFS_DIR, f"{tissue}_peptides.csv")
        df.to_csv(output_file, index=False)
        logging.info(f"Saved {len(df)} peptides from {tissue} to {output_file}")

        # Print first 10 peptides from each tissue
        print(f"\nFirst 10 peptides from {tissue}:")
        display(df.head(10))
    else:
        logging.warning("No peptides extracted from any files.")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.


Detecting adapter sequence for read1...
Detecting adapter sequence for read1...
Detecting adapter sequence for read1...
Detecting adapter sequence for read1...
No adapter detected for read1

No adapter detected for read1

No adapter detected for read1

Read1 before filtering:
total reads: 1329068
total bases: 200689268
Q20 bases: 193965942(96.6499%)
Q30 bases: 185185312(92.2746%)

Read1 after filtering:
total reads: 1324084
total bases: 199936684
Q20 bases: 193543593(96.8024%)
Q30 bases: 184878954(92.4688%)

Filtering result:
reads passed filter: 1324084
reads failed due to low quality: 4984
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 43.4827%

JSON report: fastp.json
HTML report: fastp.html

fastp -i ../backend/uploaded_files/s13_S13_L001_R1_001.fastq -o ../backend/trimmed_data/s13_S13_L001_R1_001_trimmed.fastq --detect_adapter_for_pe -q

fastp completed. Trimmed file saved to ../backend/trimmed_data/s13_S13_L001_R1_001_trimmed.fastq


Read1 before filtering:
total reads: 1541449
total bases: 232758799
Q20 bases: 225033945(96.6812%)
Q30 bases: 214951397(92.3494%)

Read1 after filtering:
total reads: 1535575
total bases: 231871825
Q20 bases: 224536629(96.8365%)
Q30 bases: 214595290(92.5491%)

Filtering result:
reads passed filter: 1535575
reads failed due to low quality: 5874
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 45.6567%

JSON report: fastp.json
HTML report: fastp.html

fastp -i ../backend/uploaded_files/s10_S10_L001_R1_001.fastq -o ../backend/trimmed_data/s10_S10_L001_R1_001_trimmed.fastq --detect_adapter_for_pe -q 20 
fastp v0.24.0, time used: 25 seconds


fastp completed. Trimmed file saved to ../backend/trimmed_data/s10_S10_L001_R1_001_trimmed.fastq


...


First 10 peptides from s10:


Sequence_ID  \
0  M06325:39:000000000-KRJ2K:1:1101:19943:1245   
1  M06325:39:000000000-KRJ2K:1:1101:21278:1282   
2  M06325:39:000000000-KRJ2K:1:1101:20362:1312   
3  M06325:39:000000000-KRJ2K:1:1101:20765:1325   
4  M06325:39:000000000-KRJ2K:1:1101:14284:1330   
5  M06325:39:000000000-KRJ2K:1:1101:19281:1340   
6  M06325:39:000000000-KRJ2K:1:1101:14233:1348   
7  M06325:39:000000000-KRJ2K:1:1101:14294:1350   
8  M06325:39:000000000-KRJ2K:1:1101:19989:1359   
9  M06325:39:000000000-KRJ2K:1:1101:20496:1359   

                                   Original_Sequence  \
0  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
1  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
2  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
3  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
4  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
5  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
6  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
7  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
8  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
9  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   

                                        DNA_Sequence         Peptide  Frame  \
0  ATGCTCGGGGATCCGAATTCTTCTCGCCGCTTTATTTGTCGCGCGT...  SRRFICRASLASSS      0   
1  ATGCTCGGGGATCCGAATTCTTCGCGCCGCCCTATTTCTCGCGCGT...  SRRPISRASLAPYP      0   
2  ATGCTCGGGGATCCGAATTCTTCTCGCCGCTTTATTCCTCGCGCGT...  SRRFIPRACLAHPF      0   
3  ATGCTCGGGGATCCGAATTCTTGGCGCCGCGTTATTCCTCGCGCGT...  WRRVIPRASLASEL      0   
4  ATGCTCGGGGATCCGAATTCTTCTCGCCGCCTTATTTCTCGCGCGT...  SRRLISRAFLAYPY      0   
5  ATGCTCGGGGATCCGAATTCTTCTCGCCGCCTTATTTATCGCGCGT...  SRRLIYRASLASFP      0   
6  ATGCTCGGGGATCCGAATTCTCATCGCCGCTTTATTTGTCGCGCGT...  HRRFICRASLAFYS      0   
7  ATGCTCGGGGATCCGAATTCTCCGCGCCGCGTTATTAATCGCGCGT...  PRRVINRAFLAFSY      0   
8  ATGCTCGGGGATCCGAATTCTTCTCGCCGCATTATTTGGCGCGCGT...  SRRIIWRALLASLP      0   
9  ATGCTCGGGGATCCGAATTCTTCTCGCCGCGTTATTTGTCGCGCTT...  SRRVICRAYLARPP      0   

  Orientation Tissue  
0           +    s10  
1           +    s10  
2           +    s10  
3           +    s10  
4           +    s10  
5           +    s10  
6           +    s10  
7           +    s10  
8           +    s10  
9           +    s10

INFO: Saved 828773 peptides from s6 to ../backend/extracted_orfs/s6_peptides.csv



First 10 peptides from s6:


Sequence_ID  \
0  M06325:39:000000000-KRJ2K:1:1101:20337:1266   
1  M06325:39:000000000-KRJ2K:1:1101:21275:1300   
2  M06325:39:000000000-KRJ2K:1:1101:12441:1338   
3  M06325:39:000000000-KRJ2K:1:1101:20360:1348   
4  M06325:39:000000000-KRJ2K:1:1101:20833:1364   
5  M06325:39:000000000-KRJ2K:1:1101:19255:1365   
6  M06325:39:000000000-KRJ2K:1:1101:21523:1371   
7  M06325:39:000000000-KRJ2K:1:1101:10992:1373   
8  M06325:39:000000000-KRJ2K:1:1101:15695:1386   
9  M06325:39:000000000-KRJ2K:1:1101:14687:1395   

                                   Original_Sequence  \
0  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
1  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
2  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
3  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
4  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
5  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
6  TGATCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
7  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
8  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   
9  GGAGCTGTCGTATTCCAGTCAGGTGTGATGCTCGGGGATCCGAATT...   

                                        DNA_Sequence  \
0  ATGCTCGGGGATCCGAATTCTTCGCGCCGCACTATTCCTCGCGCGT...   
1  ATGCTCGGGGATCCGAATTCTTCGCGCCGCGTTATTTCTCGCGCGT...   
2  ATGCTCGGGGATCCGAATTCTTGTCGCCGCGTTATTTCTCTCGCTT...   
3  ATGCTCGGGGATCCGAATTCTTGTCGCCGCGTTATTCCTCGCGCGT...   
4  ATGCTCGGGGATCCGAATTCTTGTCGCCGCCTGATTCCTCGCGCGG...   
5  ATGCTCGGGGATCCGAATTCTTGTCGCCGCCCTATTTGTCGCGCTT...   
6  ATGCTCGGGGATCCGAATTCTTGTCGCCGCCATATTTCTCGCTCTT...   
7  ATGCTCGGGGATCCGAATTCTCATCGCCGCATTATTTATCGCGCTT...   
8  ATGCTCGGGGATCCGAATTCTTATCGCCGCATTATTCCTCGCGCGA...   
9  ATGCTCGGGGATCCGAATTCTTCTCGCCGCGTTATTCCTCGCGCGA...   

                      Peptide  Frame Orientation Tissue  
0              SRRTIPRASLAYSL      0           +     s6  
1                 SRRVISRAFLA      0           +     s6  
2              CRRVISLASLASHP      0           +     s6  
3              CRRVIPRASLARPL      0           +     s6  
4              CRRLIPRAVLASVS      0           +     s6  
5              CRRPICRASLASLF      0           +     s6  
6  CRRHISRSFLSRCFYAFFPPLFPIFP      0           +     s6  
7              HRRIIYRASLASYF      0           +     s6  
8               YRRIIPRAILASS      0           +     s6  
9              SRRVIPRAILARSF      0           +     s6

## Enrichment analysis
#### Core Functionality

- Analyze sequencing results from multiple rounds (e.g., R1–R4).
- Allow input of pre/post amplification datasets for bias normalization:
  - Compute baseline ratios using pre-amplification data.
  - Adjust results post-analysis using these ratios.

#### Input Flexibility

- Users can upload a reference sequence file for specific sequences of interest to track across rounds.
- Input amplification bias ratios can be uploaded alongside pre-amplification sequences.

#### Output

- CSV with normalized enrichment ratios.
- Visualizations for Log2 fold changes and reads per million (RPM).

In [6]:
import numpy as np


def calculate_enrichment_statistics(df_list, rounds, pre_amp_data=None):
    """Calculate enrichment statistics across rounds."""
    import numpy as np
    from scipy import stats
    from statsmodels.stats.multitest import multipletests

    # Create a combined DataFrame with counts from all rounds
    combined_stats = {}

    # Initial setup
    for i, df in enumerate(df_list):
        round_name = rounds[i]
        total_reads = len(df)

        # Handle zero counts
        counts = df.groupby("Peptide")["Peptide"].count()
        rpm = (counts / total_reads) * 1e6

        if not combined_stats:
            combined_stats = {
                "Peptide": counts.index.values,
                f"RPM_{round_name}": rpm.values,
                f"Count_{round_name}": counts.values,
            }
        else:
            # Ensure all peptides are included with zero counts if missing
            combined_stats[f"RPM_{round_name}"] = rpm.reindex(
                combined_stats["Peptide"]
            ).fillna(0)
            combined_stats[f"Count_{round_name}"] = counts.reindex(
                combined_stats["Peptide"]
            ).fillna(0)

    stats_df = pd.DataFrame(combined_stats)

    # Calculate enrichment statistics
    for i in range(1, len(rounds)):
        prev_round = rounds[i - 1]
        curr_round = rounds[i]

        # Log2 fold change with pseudocount
        stats_df[f"Log2FC_{prev_round}_to_{curr_round}"] = np.log2(
            (stats_df[f"RPM_{curr_round}"] + 1) / (stats_df[f"RPM_{prev_round}"] + 1)
        )

        # Chi-square test with continuity correction
        curr_counts = stats_df[f"Count_{curr_round}"]
        prev_counts = stats_df[f"Count_{prev_round}"]
        total_curr = curr_counts.sum()
        total_prev = prev_counts.sum()

        # Add small pseudocount to avoid division by zero
        epsilon = 1e-10

        # Calculate expected frequencies
        total = total_curr + total_prev + epsilon
        expected_curr = (curr_counts + prev_counts + epsilon) * total_curr / total
        expected_prev = (curr_counts + prev_counts + epsilon) * total_prev / total

        # Chi-square statistic with Yates' correction
        chi_square = (np.abs(curr_counts - expected_curr) - 0.5) ** 2 / (
            expected_curr + epsilon
        ) + (np.abs(prev_counts - expected_prev) - 0.5) ** 2 / (expected_prev + epsilon)

        # P-values from chi-square distribution with df=1
        p_values = 1 - stats.chi2.cdf(chi_square, df=1)

        # Ensure p-values are valid
        p_values = np.clip(p_values, 1e-300, 1.0)

        stats_df[f"P_value_{prev_round}_to_{curr_round}"] = p_values

        # FDR correction
        _, q_values, _, _ = multipletests(p_values, method="fdr_bh")
        stats_df[f"Q_value_{prev_round}_to_{curr_round}"] = q_values

        # Calculate enrichment score
        stats_df[f"Enrichment_Score_{prev_round}_to_{curr_round}"] = stats_df[
            f"Log2FC_{prev_round}_to_{curr_round}"
        ] * (-np.log10(q_values + epsilon))

    return stats_df


In [7]:
import ipywidgets as widgets
from IPython.display import display
import glob
import os


# Create widgets for each round
round_widgets = []
for i in range(3):
    round_label = widgets.HTML(value=f"<b>Round {i+1}:</b>")
    file_select = widgets.Dropdown(
        description="File:",
        options=[],  # Will be populated with peptide files
        layout={"width": "500px"},
    )

    # Stack label and dropdown horizontally
    round_box = widgets.HBox([round_label, file_select])
    round_widgets.append(round_box)

# Create vertical layout of all round selectors
rounds_ui = widgets.VBox(round_widgets)


def update_file_options(change):
    # Update dropdown options based on extracted peptide files
    peptide_files = glob.glob(os.path.join(EXTRACTED_ORFS_DIR, "*_peptides.csv"))
    # Get just the filenames without path
    file_options = [os.path.basename(f) for f in peptide_files]

    for widget in round_widgets:
        widget.children[1].options = file_options


# Display the widget interface
display(rounds_ui)

# Update options when directory changes
update_file_options(None)


def get_selected_files():
    """Returns full paths to selected peptide files"""
    selected = [w.children[1].value for w in round_widgets]
    return [os.path.join(EXTRACTED_ORFS_DIR, f) for f in selected if f]


# Add confirm button
confirm_btn = widgets.Button(
    description='Confirm Selection',
    button_style='success',
    layout={'width': 'auto', 'margin': '10px 0'}
)

def on_confirm_click(b):
    files = get_selected_files()
    if not files:
        print("No files selected. Please select files before confirming.")
        return
    print("Selected files:")
    for f in files:
        print(f"- {os.path.basename(f)}")

confirm_btn.on_click(on_confirm_click)

# Add button below dropdowns
display(confirm_btn)


Button(button_style='success', description='Confirm Selection', layout=Layout(margin='10px 0', width='auto'), …

Selected files:
- s7_peptides.csv
- s10_peptides.csv
- s13_peptides.csv


In [9]:
from tqdm.auto import tqdm

selected_files = get_selected_files()
round_dfs = []
round_names = []

if selected_files:
    print("Loading selected files...")
    for file in tqdm(selected_files, desc="Loading files"):
        df = pd.read_csv(file)
        if not df.empty:
            round_dfs.append(df)
            round_names.append(os.path.basename(file).split("_")[0])

    enrichment_stats = calculate_enrichment_statistics(round_dfs, round_names)

    output_file = os.path.join(EXTRACTED_ORFS_DIR, "enrichment_statistics.csv")
    enrichment_stats.to_csv(output_file, index=False)
    print(f"\nSaved enrichment statistics to {output_file}")

    print("\nGenerating plots...")
    # Use regular print instead of display for the top enriched peptides
    print("\nTop 10 Enriched Peptides:")
    last_comparison = f"{round_names[-2]}_to_{round_names[-1]}"
    top_enriched = enrichment_stats.nlargest(10, f"Enrichment_Score_{last_comparison}")
    print(
        top_enriched[
            [
                "Peptide",
                f"Log2FC_{last_comparison}",
                f"Q_value_{last_comparison}",
                f"Enrichment_Score_{last_comparison}",
            ]
        ].to_string()
    )
else:
    print("No files selected. Please select files using the dropdown menus above.")

Loading selected files...


Loading files:   0%|          | 0/3 [00:00<?, ?it/s]


Saved enrichment statistics to ../backend/extracted_orfs/enrichment_statistics.csv

Generating plots...

Top 10 Enriched Peptides:
                       Peptide  Log2FC_s10_to_s13  Q_value_s10_to_s13  Enrichment_Score_s10_to_s13
Peptide                                                                                           
VRRSIERATLATSS  VRRSIERATLATSS           5.685907        8.692329e-14                    56.856921
QRREIERAGLATSA  QRREIERAGLATSA           5.540086        1.619723e-12                    55.362204
NRRAIQRA              NRRAIQRA           5.444951        9.391033e-12                    54.237253
NRRVITRATLANDA  NRRVITRATLANDA           5.377853        3.034697e-11                    53.159530
RRRVIDRAVLANAT  RRRVIDRAVLANAT           5.377853        3.034697e-11                    53.159530
ARRVIPRAALAKLA  ARRVIPRAALAKLA           5.302974       1.550814e-300                    53.029742
RRRIIARAELAAAS  RRRIIARAELAAAS           5.343096        5.457503e-11       